## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")


In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(
    model="llama3-8b-8192",
)

In [3]:
from langchain_core.messages import HumanMessage,SystemMessage,AIMessage
model.invoke([HumanMessage(content="Hello")])

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 11, 'total_tokens': 37, 'completion_time': 0.041761186, 'prompt_time': 0.004734133, 'queue_time': 0.32961083500000005, 'total_time': 0.046495319}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_0fb809dba3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--931deb81-294a-4d0c-9f1b-1abf7d91ab0c-0', usage_metadata={'input_tokens': 11, 'output_tokens': 26, 'total_tokens': 37})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [49]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] =ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(
    model,get_session_history
)

In [ ]:
config={"a":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hello my name is Talha Abbasi and i am learning Agentic Ai")],
    config=config
)

In [12]:
response.content

"Nice to meet you, Talha Abbasi! Congratulations on starting your journey to learn about Agent-based AI. That's a fascinating field!\n\nCan you tell me a bit more about what sparked your interest in Agentic AI? Are you looking to apply it to a specific problem or industry, or is it more of a general interest in AI and its applications?\n\nBy the way, have you started with any specific resources or courses to learn about Agentic AI? I'd be happy to help you with any questions or provide recommendations if you need them!"

In [14]:
get_session_history('chat1')

InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello my name is Talha Abbasi and i am learning Agentic Ai', additional_kwargs={}, response_metadata={}), AIMessage(content="Nice to meet you, Talha Abbasi! Congratulations on starting your journey to learn about Agent-based AI. That's a fascinating field!\n\nCan you tell me a bit more about what sparked your interest in Agentic AI? Are you looking to apply it to a specific problem or industry, or is it more of a general interest in AI and its applications?\n\nBy the way, have you started with any specific resources or courses to learn about Agentic AI? I'd be happy to help you with any questions or provide recommendations if you need them!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 25, 'total_tokens': 136, 'completion_time': 0.166400872, 'prompt_time': 0.010643316, 'queue_time': 0.419479883, 'total_time': 0.177044188}, 'model_name': 'llama3-8b-8192', 'system_fingerprin

In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="what i am doing?")],
    config=config
)

In [16]:
response.content

"You're currently learning about Agent-based AI! That's a great topic.\n\nCan you tell me more about what you're doing specifically? Are you:\n\n1. Following online courses or tutorials?\n2. Reading books or research papers on the subject?\n3. Working on a project that involves agent-based AI?\n4. Trying to build a specific application or model using agent-based AI?\n\nSharing what you're doing will help me better understand your goals and offer more targeted support and guidance."

In [17]:
config2={"configurable":{"session_id":"chat2"}}


In [ ]:
with_message_history.invoke(
    [HumanMessage(content="what i am doing?")],
    config=config2
).content

"It seems like you're asking me what you're doing. Am I correct?"

In [19]:
with_message_history.invoke(
    [HumanMessage(content="what i asked you before")],
    config=config2
).content

'Let me check... Ah yes! You asked me "what i am doing?"'

In [50]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
    [
        ('system','Your are helpful assistant answer any query in language {language}'),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [ ]:
chain= prompt | model

In [52]:
chain.invoke(
    {
        "messages": [HumanMessage(content="Helo my name is Talha Abbadi")],
        "language":"Urdu"
    }
)

AIMessage(content='مرحباً، تالہ عباسی! خوش آمدید! (Merhaba, Talha Abbasi! Khush ameedeed!) Nice to meet you, Talha Abbasi! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 34, 'total_tokens': 84, 'completion_time': 0.036520969, 'prompt_time': 0.012438086, 'queue_time': 0.6162940640000001, 'total_time': 0.048959055}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_5b339000ab', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--a2fb3f54-43b5-47f4-84fb-200abec1b41f-0', usage_metadata={'input_tokens': 34, 'output_tokens': 50, 'total_tokens': 84})

In [53]:
with_message_history=RunnableWithMessageHistory(
    chain,get_session_history,
    input_messages_key='messages'
)

In [54]:
config3={"configurable":{"session_id":"chat3"}}

In [62]:
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="my name is Talha Abbasi")],
    "language":"english"},
    config=config3
)

In [63]:
response.content

"Nice to meet you, Talha Abbasi! I'm happy to have you as a part of our conversation."

In [66]:
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="what is my name")],
    "language":"english"},
    config=config3
)

In [67]:
response.content

'I remember! Your name is Talha Abbasi!'

# Managing Conversational History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in. 

<b>'trim_messages'</b> helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [94]:
from langchain_core.messages import HumanMessage,SystemMessage,AIMessage,trim_messages

trimmer=trim_messages(
    max_tokens=100,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [95]:
messages = [
    SystemMessage("you're a good assistant, you always respond with a joke."),
    HumanMessage("i wonder why it's called langchain"),
    AIMessage(
        'Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!'
    ),
    HumanMessage("and who is harrison chasing anyways"),
    AIMessage(
        "Hmmm let me think.\n\nWhy, he's probably chasing after the last cup of coffee in the office!"
    ),
    HumanMessage("what do you call a speechless parrot"),
]

In [96]:
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant, you always respond with a joke.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="i wonder why it's called langchain", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='and who is harrison chasing anyways', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hmmm let me think.\n\nWhy, he's probably chasing after the last cup of coffee in the office!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what do you call a speechless parrot', additional_kwargs={}, response_metadata={})]

In [100]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(
        messages=itemgetter("messages")| trimmer)| prompt | model
    
)

In [102]:
chain.invoke({
    "messages": messages + [HumanMessage(content="who is harrison")],
    "language": "english"
}).content


"You're referring to Harrison Ford, right?\n\nWell, I think Harrison Ford is probably chasing after his next great adventure... or maybe just trying to navigate his way out of a Star Wars plot twist!"

In [105]:
# lets wrap it with message history
with_message_history=RunnableWithMessageHistory(
    chain,get_session_history,
    input_messages_key='messages'
)
config6={"configurable":{"session_id":"chat7"}}

In [107]:
with_message_history.invoke(
    {
    "messages": messages + [HumanMessage(content="who is harrison")],
    "language": "english"
    },
    config=config6
).content

'You\'re referring to Harrison Ford\'s character, Indiana Jones, and his infamous chase in the movie Raiders of the Lost Ark!\n\nBut, to answer your original question, Harrison Ford\'s character in the movie "Blade Runner" is Rick Deckard, and he\'s chasing after... (drumroll please)... Replicants!'